# Applied Deep Learning Tutorial 
contact: Mark.schutera@kit.edu


# Recurrent Neural Networks (RNNs) for Text Generation

## Introduction
In this tutorial, you will attempt to implement a recurrent neural network for text generation based on a text dataset from Shakespeare / Goethe. The tutorial has been inspired by the original [tensorflow tutorials](www.tensorflow.org) and Andrej Karpathies work and thus based on numpy only. 

<img src="graphics/GoetheSchillerWeimar.jpg" width="700"><br>
<center> Fig. 1: The german poets Goethe and Schiller in Weimar. Image from [pixabay](https://pixabay.com/de/photos/weimar-goethe-schiller-denkmal-806851/) </center>


## Core Idea 

In this tutorial we will focus on a text generation approach based on a character-based RNN. We will work with a dataset similar to the dataset of Shakespeare's writing provided by Andrej Karpathy's and first used in his [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Mephist"), train a model to predict the next character in the sequence ("o"). Longer sequences of text can be generated by calling the model repeatedly, or in other words recurrent.


In [ ]:
import numpy as np
import string

## Scraping Text Data

First we need to scrape data. A good source to scrape is [Project Gutenberg](https://www.gutenberg.org/).
Since there are legal struggles with project gutenberg we will use Faust 1 by Goethe, scraped from [wikisource](https://de.wikisource.org/wiki/Faust_-_Der_Tragödie_erster_Teil). Or shakespeare already scraped for us by Google.


In [ ]:
# Download the data
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Link to Goethe data
path_to_file = 'data/Faust1_Goethe.txt'



In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

In [ ]:
# Look at the first 1000 characters
print(text[:1000])

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Print the vocabulary
print(vocab)


In [ ]:
# Delete unwanted characters
import re
char_list = '''select characters to clean the dataset'''
text = re.sub("|".join(char_list), '''substitute with an empty string''', text)

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Print the vocabulary
print(vocab)

# Look at the first 1000 characters
'''print the first 1000 characters of the text'''

## Vectorize the characters
Before training the strings need to be vectorized, meaning we need to bring them in a numerical representation, for our neural network to be able to work with. 



In [ ]:
# Creating a mapping from unique characters to indices
char_to_ix = {u:i for i, u in enumerate(vocab)}
ix_to_char = np.array(vocab)

text_as_int = np.array('''transpose all characters of the text to ints''' for c in text])

# Visualize the mapping
print('{')
for char,_ in zip(char_to_ix, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_to_ix[char]))
print('  ...\n}')

## Sample the data
The task of the neural network is a prediction task. Given a sequence of characters what is the most probable next character. 
We can then move our receptive field one stride forward and the neural network again performs a prediction for the next character. The output will be our input shifted one character to the right. 

For this we use the [from_tensor_slices](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices) utility of tensorflow.

In [ ]:
# Prepare the input dimensions
# Make a reasonable choise for the sequence length, elaborate. 
seq_length = 100
vocab_size = len(vocab)


def sample(h, seed_ix, n):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []

    # How is the following pipeline also called? It is the ... of the model.
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

## Designing our Recurrent Neural Network

In [ ]:
# Designing our Recurrent Neural Network

# Make a reasonable choise for the learning rate, elaborate. 
learning_rate = 1e-3
hidden_size = 100

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01  # input to hidden
Whh = '''design the weight matrix for the hidden layer'''
Why = np.random.randn(vocab_size, hidden_size) * 0.01  # hidden to output
bh = np.zeros((hidden_size, 1))  # hidden bias
by = np.zeros((vocab_size, 1))  # output bias


## Defining the loss function for the model



In [ ]:
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t - 1]) + bh)  # hidden state
        ys[t] = np.dot(Why, hs[t]) + by  # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))  # probabilities for next chars
        loss += -np.log(ps[t][targets[t], 0])  # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])

    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext  # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t - 1].T)
        dhnext = np.dot(Whh.T, dhraw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)  # clip to mitigate exploding gradients

    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]

## Train Model

In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0

while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(text) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0  # go from start of data

    inputs = [char_to_ix[ch] for ch in text[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in text[p + 1:p + seq_length + 1]]
   
    # sample from the model now and then
    if n % 1000 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    if n % 1000 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss))  # print progress
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update
    p += seq_length  # move data pointer
    n += 1  # iteration counter

## Judging our Text Generator

When judging the results keep in mind our prerequisits.
Following is an example of 1000 characters of a model trained for 100 epochs.

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:
- The model is character-based. When training started, the model did not know how to spell a single word, let alone german word, or that words were even a unit of text.
- The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.
- As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Next steps to take it from here

- Write a function to capture the current state of the model, to be able to save and later continue the training. How would you go about it (mail to mark.schutera@kit.edu for a chance to get bonus points). 
- Scrape Kafka or Goethe and try training on a different dataset. What are your findings - can you distinguish between generated Shakespeare and generated Kafka?
- So far our approach is character based, try a word based approach, what are the advantages, and what are the drawbacks? Which approach would you choose with respect to dataset size, computational power, and overall performance?
